In [30]:
import sys, os
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy import optimize
import pickle
from time import gmtime, strftime
import generic_potential_modified
#from cosmoTransitions import tunneling1D
import tunneling1D_modified as tunneling1D
#from cosmoTransitions import pathDeformation as pd

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout = self._original_stdout

# scale-free potential without a window function:
def get_f_max_and_width(b, c):
    T_c = 1/np.sqrt(2*(c - b**2))
    phi_minus = (-3*b*T_c - np.sqrt(9*b**2*T_c**2 + 4*(1 - 2*c*T_c**2)))/2
    Vtot = (-.5 + c*T_c**2)*phi_minus**2 + b*T_c*phi_minus**3 + .25*phi_minus**4
    f_max = (Vtot/T_c**4)/10
    width = 2*phi_minus
    return f_max, width

def get_fs_and_widths(b, c, numf):
    bs = np.tile(b, (c.size, 1)).T
    cs = np.tile(c, (b.size, 1))
    f_maxs, widths = get_f_max_and_width(bs, cs)
    fs = np.linspace(-f_maxs, f_maxs, numf).T
    return fs, widths

############################################################################################

# Potential parameters that we want to scan over
#bvals = np.linspace(-4, -0.01, num=20)
#cvals = np.linspace(0, 16, num=20)
bvals = np.linspace(-2.0001, -2, num=2)
cvals = np.linspace(9.9999, 10, num=2)
Lamvals = np.array([1.])
c1vals = np.array([7.])
numf = 15
fvals, widthvals = get_fs_and_widths(bvals, cvals, numf)

# Fixed parameters
gstar = 100

# params fed into potential
params = {
    'a': 0,
    'b': bvals,
    'c': cvals,
    'Lambda': Lamvals,
    'c1': c1vals,
    'f':fvals,
    'width': widthvals
}

#controlls if any and how many b and c values should be retried around failed points
recalc = True
recalc_all = False
recalc_resolution = 3
############################################################################################

class model1(generic_potential_modified.generic_potential):
    """
    A sample model which makes use of the *generic_potential* class.

    This model doesn't have any physical significance. Instead, it is chosen
    to highlight some of the features of the *generic_potential* class.
    It consists of two scalar fields labeled *phi1* and *phi2*, plus a mixing
    term and an extra boson whose mass depends on both fields.
    It has low-temperature, mid-temperature, and high-temperature phases, all
    of which are found from the *getPhases()* function.
    """
    def init(self, a, b, c, Lambda, c1, f, width, **kwargs):
        """
          m1 - tree-level mass of first singlet when mu = 0.
          m2 - tree-level mass of second singlet when mu = 0.
          mu - mass coefficient for the mixing term.
          Y1 - Coupling of the extra boson to the two scalars individually
          Y2 - Coupling to the two scalars together: m^2 = Y2*s1*s2
          n - degrees of freedom of the boson that is coupling.
        """
        # The init method is called by the generic_potential class, after it
        # already does some of its own initialization in the default __init__()
        # method. This is necessary for all subclasses to implement.

        # This first line is absolutely essential in all subclasses.
        # It specifies the number of field-dimensions in the theory.
        self.Ndim = 1
        self.deriv_order = 2
        self.Tmax = 2.

        # This next block sets all of the parameters that go into the potential
        # and the masses. This will obviously need to be changed for different
        # models.
        self.a = a
        self.b = b
        self.c = c
        self.Lambda = Lambda
        self.c1 = c1
        self.f = f
        self.width = width
        
    def approxZeroTMin(self):
        """
        Returns approximate values of the zero-temperature minima.
        This should be overridden by subclasses, although it is not strictly
        necessary if there is only one minimum at tree level. The precise values
        of the minima will later be found using :func:`scipy.optimize.fmin`.
        Returns
        -------
        minima : list
            A list of points of the approximate minima.
        """
        # This should be overridden.
        #CHRISTY NOTE: The origional code uses the default self.renormScaleSq = 1000.**2 as the approxZeroTMin. This is the default scale used in the MS-bar renormalization. Our minima are at much smaller scales, and so this function is being overwritten
        #return [np.ones(self.Ndim)*self.renormScaleSq**.5]
        
        approxZeroGuess = 1.**2
        return [np.ones(self.Ndim)*approxZeroGuess**.5]

    def Vtot(self, X, T, include_radiation=True):
        """
        This method defines the tree-level potential. It should generally be
        subclassed. (You could also subclass Vtot() directly, and put in all of
        quantum corrections yourself).
        """
        # X is the input field array. It is helpful to ensure that it is a
        # numpy array before splitting it into its components.
        X = np.asanyarray(X)
        # x and y are the two fields that make up the input. The array should
        # always be defined such that the very last axis contains the different
        # fields, hence the ellipses.
        # (For example, X can be an array of N two dimensional points and have
        # shape (N,2), but it should NOT be a series of two arrays of length N
        # and have shape (2,N).)
        phi = X[...,0]
        
        # JACK NOTE: Here I wanted to avoid checking parameter space that we knew was not going to produce a 1st order PT
#         assert C < 0.5 + 9*B**2/(8*(1+4*A)), "no symmetry-breaking minimum"
#         assert C/(B+1e-12)**2 > 1, f"no phase transition; A {A}, B {B}, C {C}"
        w = (1-1/(1+np.exp(-self.c1*(phi-self.width/2))))/(1+np.exp(-self.c1*(phi+self.width/2)))
        w_0 = (1-1/(1+np.exp(-self.c1*(0-self.width/2))))/(1+np.exp(-self.c1*(0+self.width/2)))
        return ((-0.5 + self.c * T**2)*phi**2 + self.b * T*phi**3 + (0.25 + self.a)*phi**4 + self.f * T**4 * (w-w_0))
    
def get_profile(phi_vals, V_vals, guesses=(5, 0)):
    f = interpolate.UnivariateSpline(phi_vals, V_vals, s=0, k=4)
    df = f.derivative(1)
    d2f = f.derivative(2)
    inst = tunneling1D.SingleFieldInstanton(*guesses, f, df, d2f)
    profile = inst.findProfile()
    action = inst.findAction(profile)
    
    return profile, action

In [31]:
# Calculate action for field defined in param_instance, at all nucleation temps and temps in other_temps
other_temps = [0.25, 0.26, 0.27, 0.275]
b_inst, c_inst = -2, 10
fmax, width = get_f_max_and_width(b_inst, c_inst)
param_instance = {
    'a': 0,
    'b': b_inst,
    'c': c_inst,
    'Lambda': 1.,
    'c1': 7.,
    'f':.65*fmax,
    'width': width
}
m = model1(**param_instance)
with HiddenPrints():
    m.findAllTransitions(tunnelFromPhase_args={'verbose': False, 'fullTunneling_params': {'verbose': False}})
print(f'Number of transitions found: {np.size(m.TnTrans)}\n')
Tnucs = [m.TnTrans[i]['Tnuc'] for i in range(np.size(m.TnTrans))]
high_vevs = [m.TnTrans[i]['high_vev'][0] for i in range(np.size(m.TnTrans))]
low_vevs = [m.TnTrans[i]['low_vev'][0] for i in range(np.size(m.TnTrans))]
for i in range(np.size(m.TnTrans)):
    phis = np.linspace(high_vevs[i] - np.abs(high_vevs[i]), low_vevs[i] + np.abs(low_vevs[i]), num=10000)
    V_vals = m.Vtot(np.array([phis]).T, Tnucs[i])
    guesses = (low_vevs[i] + 0.01*np.abs(low_vevs[i]), high_vevs[i] - 0.01*np.abs(high_vevs[i]))
    profile, action = get_profile(phis, V_vals, guesses=guesses)
    print(f'#####Transition {i+1}#####')
    print(f'Nucleation temp: {Tnucs[i]}')
    print(f'high vev -> low vev: {high_vevs[i]} -> {low_vevs[i]}')
    print(f'S: {action}')
    for ot in other_temps:
        V_vals = m.Vtot(np.array([phis]).T, ot)
        profile, action = get_profile(phis, V_vals, guesses=guesses)
        print(f'S at temp({ot}): {action}')

Number of transitions found: 1

#####Transition 1#####
Nucleation temp: 0.2751712502382193
high vev -> low vev: -1.7347234759765717e-18 -> 1.2343804417277089
S: 39.23421142210297
S at temp(0.25): 4.386007616378241
S at temp(0.26): 9.626726936206381
S at temp(0.27): 22.34386709344223
S at temp(0.275): 38.54863415793112
